In [0]:
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmparentchildhierarchy`
  WHERE
    adjacencypath like 'Entity|Shared_Mbr_Entity_Structure|Management|BU_NA|%'
  and IsBaseRecord = 1
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
voluntaryturnover as (
  SELECT
    `Calendar Date`,
    branchid,
    branchname,
    branchcountry,
    sum(ftevoluntaryturnover) as turnover_all
  FROM
    (
      SELECT
        TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
        vt.epmbranch as branchid,
        branch.alias as branchname,
        vt.Country as branchcountry,
        costtype,
        ftevoluntaryturnover
      FROM
        `NA-CDP-BU-HR-DEV`.curated.voluntaryturnover as vt
          LEFT JOIN branchnames as branch
            on branch.backendname = vt.EPMBranch
      WHERE vt.EPMBranch NOT IN ('100020','100023')
    )
  GROUP BY
    `Calendar Date`,
    branchid,
    branchname,
    branchcountry
)
--SELECT * FROM voluntaryturnover where Year(`Calendar Date`) = 2025
SELECT
  *
FROM
  voluntaryturnover
  WHERE  `Calendar Date` >= make_date(2023,10,1)
